# Laboratorium 1 - A* vs Dijkstra / Tabu search
### Lukasz Fabia 272724


### Jak zacząć?

Python3, najlepiej wersja > 3.11.

```bash
python -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt
```

## Teoria i cel

Celem tego zdania jest znalezienie najkrótszego (min. droga) przejazdu MPK lub przejechanie z punktu A do B w najkrótszym czasie (min. czasu).

Do tego problemu optymalizacji należy wykorzystać dwa popularne algorytmy do wyszukiwania najktrótszych ścieżek - A* i Dijkstra. 

---

### Dijkstra

**Algorytm zachłanny**, znajdujący najkrótszą ścieżkę od węzła startowego do wszystkich innych węzłów w grafie. Wykorzstuje wagi (u mnie i w zadaniu korzysta on z kosztu czasu).

### A*

Jest sprowadzalny do **Dijkstry**. Można powiedzieć, że jest taką ulepszoną wersją i jest częściej używany ze względu na optymalność. A* korzysta z heurystyk, żeby oszacować, czy opłaca się wybierać taka a nie inną ścieżkę. Warto dodać, że działa to gdy wiemy czego szukamy, czyli znany jest _target_.

## Wgląd do danych

Pierwsze 5 wierszy. Z racji tego, że jednym z czynników, którym będziemy się kierować przy wybieraniu trasy będzie czas no to warto różnicę, która będzie kosztem danej ścieżki.

In [1]:
from parser import get_df
df = get_df()

df.head()

,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,MPK Autobusy,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,MPK Autobusy,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,MPK Autobusy,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,MPK Autobusy,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,MPK Autobusy,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383


## Jak zbudowałem strukturę grafu?

Generalnie w `/models/graph.py` mamy całą strukturę składa się ona z `Node`, `Edge` i coś co agreguje w sobie węzły, czyli graf. W klasie grafu znajduje się tylko słownik [ulica: węzeł]. 

Kolejnym krokiem było parsowanie danych i wrzucenie ich do grafu. Iterowałem po wierszach i:

1. Wyciągałem dane z wiersza i robiłem z tego krawędź.

2. Dodawałem startowy i końcowy przystanek do słownika. Dodawanie działa w taki sposób, że jeśli ulicy nie ma w kluczach to dodaje dane tej ulicy pod tym kluczem.

3. Aktualizacja krawędzi, czyli do listy startowego węzła dodaje nowy edge z podpiętymi węzłami.


W ten sposób, wystarczy utworzyć obiekt i otrzymujemy `Graf skierowany ważony`. Gdzie **waga** to czas przejazdu w minutach z A do B, a **skierowany** dlatego, że dodaje nową krawędź do startowego node'a. Więcej na temat struktury w kodzie źródłowym [models/graph.py](models/graph.py).

Ostatnia uwaga, przyjąłem, że przy szukaniu połączenia (krawędzi) mogę czekać maksymalnie **15 minut** (czyli od teraz + 15 minut), aby ograniczyć liczbę bezsensownych połączeń. Przykład: jestem na Grunwaldzie, a algorytm proponuje mi połączenie, które odjeżdża za godzinę, ale podróż trwa tylko **12 minut do celu**. Taka optymalizacja.

## Strategie działania

Ten problem można fajnie rozwiązać za pomoca strategii. Logika leży tylko w wybieraniu najbardziej **optymalnej** ścieżki. Zatem zbudowałem sobie szkielet z częścią wspólną tych algorytmów i będę manipulować tylko w konkretnych implementacjach wyborem ścieżki. W ten sposób będzie można tworzyć łatwiej nowe odmiany tych algorytmów.

**Odpowiedź** na konieć to jest czas w formacie `HH:MM:SS`, żeby było czytelniej, _liczba odwiedzonych wierzchołków_ i _czas wykonania_(w ms).

### Zadanie 1a 

Algorytm wyszukiwania najkrótszej ścieżki z A do B za pomocą algorytmu Dijkstry w oparciu o kryterium czasu.

### Inicjalizacja grafu

In [2]:
from parser import to_graph
from dijkstra import Dijkstra
from a import AStarMinTime, AStarMinTransfers, AStarModified
from datetime import time


g = to_graph()

In [3]:
# Times
t1 = time(hour=8, minute=18, second=0)
t2 = time(hour=20, minute=50, second=0)
t3 = time(hour=7, minute=20, second=0)

#### Wnioski

1. Minimalny czas podróży
    
    - gwarantuje nam, że koszt faktycznie będzie najmniejszy w moim przypadku - czas podróży, ale gdy warunki będą idealne tj. brak opóźnień tramwajów.

    - można pomyśleć nad algorytmem, który obsłuży sytuacje losowe.

2. Liczba odwiedzonych wierzchołków

    - jest ona całkiem spora, gdy w gre wchodzi trudniejsza trasa na odcinku **Wyszyńskiego** - **Wielka** odwiedził aż _10323_.

3. Performance

    - długi czas wykonania się dla tras gdzie mamy kilka przystanków to kilkanaście ms, ale liczba rośnie w przypadku trudniejszych odcinków. Jest to spowodowane przeszukiwaniem wszerz co też nijako wiąże się z dużą ilością odwiedzonych wierzchołków. 

    - sam język nie jest demonem prędkości, można przepisać na coś szybszego jak (np. Rust, C/C++).

4. Optymalizacje

    - użycie algorytmu z heurystyką np. A*


In [4]:
d_engine = Dijkstra(g)

# _ = d_engine.search("Muchobór Wielki", "Mroźna", t1)
_ = d_engine.search("Zajezdnia Obornicka", "Bałtycka", t2)
# _ = d_engine.search("Wyszyńskiego", "PL. GRUNWALDZKI", t3)

It took: 5.79 ms

Results for Dijkstra:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
----------------------------------------------------------------------------------------------------
A     20:52:00    Zajezdnia Obornicka           20:53:00    Paprotna                       3         
A     20:53:00    Paprotna                      20:54:00    Obornicka (Wołowska)           4         
A     20:54:00    Obornicka (Wołowska)          20:55:00    Bezpieczna                     5         
A     20:55:00    Bezpieczna                    20:57:00    Bałtycka                       7         
----------------------------------------------------------------------------------------------------

Total time (Dijkstra): 00:07:00
Total visited nodes: 250



### Zadanie 1b 
Algorytm wyszukiwania najkrótszej ścieżki z A do B za pomocą algorytmu A* w oparciu o kryterium czasu.

Jako, że A* to jest modyfikacja Dijkstry to po prostu wartość `priority` to będzie strategia obliczania kosztu dla mojej klasy w tym wypadku klasycznego A*. Poniżej snipped kodu z [a.py](a.py) z liczeniem priorytetu.

```python
priority = self.cost_strategy(
              new_cost=new_cost,
              end_node=end_node,
              next_end_node=next_edge.end_node,
          )
```

A tutaj heurystyka, zastosowana w szukaniu obiecującej ścieżki. Akturat do policzenia odległość użyłem biblioteki `geopy` ze względu na współrzędne geograficzne. Wszystkie heurystyki znajdują się w [search.py](search.py).

```python
def cost_strategy(self, new_cost, **kwargs):
    return new_cost + self._geo_heuristic(
        a=kwargs["end_node"], b=kwargs["next_end_node"]
    )
```


#### Wnioski

1. Minimalny czas podróży

    - rozwiązanie jest gorsze jak można było się spodziewać, ale bardzo szybko je dostajemy

    - warto dodać, że jest to zależne od heurystyki
    

2. Liczba odwiedzonych wierzchołków

    - liczba jest o wiele mniejsza od rozwiązania problemu za pomocą **Dijkstry**

    - jest to spowodowane użyciem heurystyki, która przeszukuje rokujące węzły

    - przy trasie najdłuższej odwiedzonych było **735** wierzchołków, gdzie w poprzednim teście przy wykorzystaniu **Dijkstry** było to _10323_.


3. Performance

    - tu jest znaczenie lepiej bo nie przeszukujemy wszystkiego

4. Opytmalizacje

    - można dodać heurystyke, która by sprawdzała kierunek, którym ma się kierować na podstawie kątów obliczanych z koordynatów punktu A i B.   

In [5]:
a_engine = AStarMinTime(g)
print(t2)
_ = a_engine.search("Muchobór Wielki", "Mroźna", t1)
_ = a_engine.search("Zajezdnia Obornicka", "Bałtycka", t2)
_ = a_engine.search("Wyszyńskiego", "PL. GRUNWALDZKI", t3)

20:50:00
It took: 1159.29 ms

Results for AStarMinTime:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
----------------------------------------------------------------------------------------------------
119   08:21:00    Muchobór Wielki               08:22:00    Stanisławowska (W.K. Forma     4         
119   08:22:00    Stanisławowska (W.K. Forma    08:23:00    Trawowa                        5         
119   08:23:00    Trawowa                       08:24:00    Krzemieniecka                  6         
119   08:24:00    Krzemieniecka                 08:25:00    Końcowa                        7         
119   08:25:00    Końcowa                       08:27:00    Ostrowskiego                   9         
119   08:27:00    Ostrowskiego                  08:28:00    FAT                            10        
14    08:30:00    FAT                           08:32:00    Aleja Pracy                    14        
14    08:32:00    Aleja Prac

#### Zadanie 1c

**Minimalizacja liczby przesiadek**. Intuicja podpowiada, że trzeba będzie wprowadzić coś w stylu kary, dodatkowego kosztu doliczanego do ścieżki, która może i jest najkrótszą, ale wymaga przesiadki właśnie. W ten sposób algorytm będzie brał pod uwagę ścieżki, które nie wymagają zmiany `line`. Tak naprawdę cała zabawa sprowadziła się do dodatania do nowego kosztu sprawdzienie ile kosztować przesiadka. Poniżej _snipped_ z source kodu, który znajduje się [a.py](a.py). 

```python
new_cost = (
            cost_so_far[current_node]
            + self.graph.compute_cost(next_edge, current_time)
            + self.graph.line_change_cost(
                edge=came_from[current_node.name], next_edge=next_edge
            )
        )
```


#### Wnioski

Wynik jest podobny do zwykłego A*, gdzie minimalizowany był czas. Tutaj mieliśmy minimalizować przesiadki i powiedzmy, że się udało na przykładowych trasach. Mamy mniej zmian linii, czas nieznaczenie się wydłużył i widać doliczoną karę za przesiadkę. W niektórych testach liczba odwiedzonych wierzchołków jest mniejsza. 

In [6]:

a_mut_engine = AStarMinTransfers(g)

_ = a_mut_engine.search("Muchobór Wielki", "Mroźna", t1)
_ = a_mut_engine.search("Zajezdnia Obornicka", "Bałtycka", t2)
_ = a_mut_engine.search("Wyszyńskiego", "PL. GRUNWALDZKI", t3)

It took: 1152.04 ms

Results for AStarMinTransfers:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
----------------------------------------------------------------------------------------------------
119   08:21:00    Muchobór Wielki               08:22:00    Stanisławowska (W.K. Forma     4         
119   08:22:00    Stanisławowska (W.K. Forma    08:23:00    Trawowa                        5         
119   08:23:00    Trawowa                       08:24:00    Krzemieniecka                  6         
119   08:24:00    Krzemieniecka                 08:25:00    Końcowa                        7         
119   08:25:00    Końcowa                       08:27:00    Ostrowskiego                   9         
119   08:27:00    Ostrowskiego                  08:28:00    FAT                            10        
14    08:30:00    FAT                           08:32:00    Aleja Pracy                    14        
14    08:32:00    Aleja Pracy   

# Podsumowanie pierwszej części listy

Generalnie zgodnie z założeniem Dijkstra zwraca najlepsze wyniki w nie najlepszym czasie. Udało się w miarę sensownie zaimplementować A* (minimalizacja czasu albo przystanków), który zwaraca dobrą odpowiedź w naprawdę fajnym czasie. Udało się napisać względnie bez duplikacji kodu przez co łatwo można pisać swoje implementacje, które jakoś optymalizują wybór ścieżki. 

Największy problem jednak sprawił sam `Python`, jako osoba, która raczej jest przyzwyczajona do **Go**, **TypeScripta**, **Javy** no to było ciężko debugować, ale trzeba przyznać, że _JupyterNotebook_ pomógł w procesie. Last but not least, ułatwieniem było, że doba nie trwała 24h tylko więcej to pomogło się skupiać na policzeniu kosztu w minutach między węzłami (przystankami).    




## Tabu Search

**Metaheurystyka** polegająca na interacyjnym przeszukiwaniu sąsiedztwa, uwzględnia zestaw ruchów niedozwolonych. Wymaga dodakowej pamięci w porównaniu do **przeszukiwania lokalnego**.

W zadaniu należy przechejać przez wszystkie przystanki i wrócić się do punktu początkowego po przez minimalizację:

- czasu

- przesiadek

Podobnie jak w poprzednim zadaniu tutaj też postarałem się aby nie duplikować kodu i poprostu parametry w zadaniu będę przyjmować opcjonalnie.

W implementacji `wycieczki` po mieście użyłem A*, ponieważ Dijkstra wykonywałby się znacznie dłużej.


## Przykład użycia

In [7]:
from tabu import Tabu
max_iter = 111
points = ["PL. GRUNWALDZKI", "Dubois", "DWORZEC GŁÓWNY"]
src = "Wyszyńskiego"
t = Tabu(g=g, t=t2, points=points, src=src, max_iter=max_iter)

In [8]:
best_sln = t.search()
print(f'Best solution {best_sln}')
t.set_points(best_sln)
t.go_on_a_trip()


It took: 1640.09 ms
Best solution ['Wyszyńskiego', 'DWORZEC GŁÓWNY', 'PL. GRUNWALDZKI', 'Dubois']
It took: 0.01 ms

Results for AStarMinTime:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

Total time (AStarMinTime): 00:00:00
Total visited nodes: 0

It took: 149.63 ms

Results for AStarMinTime:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
----------------------------------------------------------------------------------------------------
N     20:50:00    Wyszyńskiego                  20:51:00    Damrota                        1         
N     20:51:00    Damrota                       20:54:00    KROMERA                        4         
D     20:56:00    KROMERA                 

### Tabu Search dobór długości tablicy T (b)

Dostosowuje długość listy tabu w trakcie działania, w zależności od postępu algorytmu. Długość T to 10, ale można zmienić. 

In [9]:
best_sln = t.dynamic_search()
print(f'Best solution {best_sln}')
t.set_points(best_sln)
t.go_on_a_trip()


It took: 3340.17 ms
Best solution ['Wyszyńskiego', 'Wyszyńskiego', 'Dubois', 'DWORZEC GŁÓWNY', 'PL. GRUNWALDZKI']
It took: 0.01 ms

Results for AStarMinTime:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

Total time (AStarMinTime): 00:00:00
Total visited nodes: 0

It took: 0.00 ms

Results for AStarMinTime:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

Total time (AStarMinTime): 00:00:00
Total visited nodes: 0

It took: 118.73 ms

Results for AStarMinTime:
Line  Departure   Start 

### Tabu Search aspiracja (c)

Pozwala zaakceptować rozwiązania z listy tabu, jeśli są lepsze niż poprzednie najlepsze rozwiązanie.

In [10]:
best_sln = t.aspiration_search()
print(f'Best solution {best_sln}')
t.set_points(best_sln)
t.go_on_a_trip()


It took: 6504.04 ms
Best solution ['Wyszyńskiego', 'Wyszyńskiego', 'Dubois', 'PL. GRUNWALDZKI', 'DWORZEC GŁÓWNY', 'Wyszyńskiego']
It took: 0.01 ms

Results for AStarMinTime:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

Total time (AStarMinTime): 00:00:00
Total visited nodes: 0

It took: 0.00 ms

Results for AStarMinTime:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

Total time (AStarMinTime): 00:00:00
Total visited nodes: 0

It took: 121.98 ms

Results for AStarMinTime:
Line  De

### Tabu Search sampling (d) 

Wybieram losowo k list z sąsiadami, w celu zwiększenia różnorodności poszukiwań


In [11]:
best_sln = t.sampling_search()
print(f'Best solution {best_sln}')
t.set_points(best_sln)
t.go_on_a_trip()


It took: 10648.64 ms
Best solution ['Wyszyńskiego', 'Wyszyńskiego', 'DWORZEC GŁÓWNY', 'Wyszyńskiego', 'Dubois', 'PL. GRUNWALDZKI', 'Wyszyńskiego']
It took: 0.01 ms

Results for AStarMinTime:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

Total time (AStarMinTime): 00:00:00
Total visited nodes: 0

It took: 0.00 ms

Results for AStarMinTime:
Line  Departure   Start Stop                    Arrival     End Stop                      Cost      
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

Total time (AStarMinTime): 00:00:00
Total visited nodes: 0

It took: 142.84 ms

Results for AStar